In [1]:
import urllib3
import requests

In [8]:
import requests
import json
import datetime
import csv
import time

#app_id = "<FILL IN>"
#app_secret = "<FILL IN>" # DO NOT SHARE WITH ANYONE!
page_id = input("Please Paste Public Page Name:")

#access_token = app_id + "|" + app_secret

access_token = input("Please Paste Your Access Token:")
start_date = input("Please enter the start date (yyyy-mm-dd)")
end_date = input("Please enter the end date (yyyy-mm-dd)")



def request_until_succeed(url):
    response = requests.get(url)
    return response.text

# Needed to write tricky unicode correctly to csv
def unicode_normalize(text):
    return text.translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 0x201D:0x22,
                            0xa0:0x20 }).encode('utf-8')

def getFacebookPageFeedData(page_id, access_token, num_statuses):

    # Construct the URL string; see http://stackoverflow.com/a/37239851 for
    # Reactions parameters
    base = "https://graph.facebook.com/v2.12"
    node = "/%s/posts" % page_id 
    fields = "/?fields=message,link,permalink_url,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions" + \
            ".limit(0).summary(true)"
    parameters = "&limit=%s&access_token=%s&since=%s&until=%s" % (num_statuses, access_token, start_date, end_date)
    url = base + node + fields + parameters

    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data

def getReactionsForStatus(status_id, access_token):

    # See http://stackoverflow.com/a/37239851 for Reactions parameters
        # Reactions are only accessable at a single-post endpoint

    base = "https://graph.facebook.com/v2.6"
    node = "/%s" % status_id
    reactions = "/?fields=" \
            "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
            ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
            ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
            ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
            ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
            ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
    parameters = "&access_token=%s" % access_token
    url = base + node + reactions + parameters

    # retrieve data
    data = json.loads(request_until_succeed(url))
     
    return data


def processFacebookPageFeedStatus(status, access_token):

    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.

    # Additionally, some items may not always exist,
    # so must check for existence first

    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else \
            unicode_normalize(status['message'])
    link_name = '' if 'name' not in status.keys() else \
            unicode_normalize(status['name'])
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else \
            unicode_normalize(status['link'])
    status_permalink_url = '' if 'permalink_url' not in status.keys() else \
            unicode_normalize(status['permalink_url'])
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.

    status_published = datetime.datetime.strptime(
            status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + \
            datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime(
            '%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs

    # Nested items require chaining dictionary keys.

    num_reactions = 0 if 'reactions' not in status else \
            status['reactions']['summary']['total_count']
    num_comments = 0 if 'comments' not in status else \
            status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status else status['shares']['count']

    # Counts of each reaction separately; good for sentiment
    # Only check for reactions if past date of implementation:
    # http://newsroom.fb.com/news/2016/02/reactions-now-available-globally/

    reactions = getReactionsForStatus(status_id, access_token) if \
            status_published > '2016-02-24 00:00:00' else {}

    num_likes = 0 if 'like' not in reactions else \
            reactions['like']['summary']['total_count']

    # Special case: Set number of Likes to Number of reactions for pre-reaction
    # statuses

    num_likes = num_reactions if status_published < '2016-02-24 00:00:00' \
            else num_likes

    def get_num_total_reactions(reaction_type, reactions):
        if reaction_type not in reactions:
            return 0
        else:
            return reactions[reaction_type]['summary']['total_count']

    num_loves = get_num_total_reactions('love', reactions)
    num_wows = get_num_total_reactions('wow', reactions)
    num_hahas = get_num_total_reactions('haha', reactions)
    num_sads = get_num_total_reactions('sad', reactions)
    num_angrys = get_num_total_reactions('angry', reactions)

    # Return a tuple of all processed data

    return (status_id, status_message, link_name, status_type, status_link, status_permalink_url,
            status_published, num_reactions, num_comments, num_shares,
            num_likes, num_loves, num_wows, num_hahas, num_sads, num_angrys)


def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'w') as file:
        w = csv.writer(file)
        w.writerow(["status_id"])

        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        print("TYPE: ", type(scrape_starttime))
        #scrape_starttime = '2017-04-03'
        print ("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))

        statuses = getFacebookPageFeedData(page_id, access_token, 100)

        while has_next_page:
            for status in statuses['data']:

                # Ensure it is a status with the expected metadata
                if 'reactions' in status:
                    w.writerow(processFacebookPageFeedStatus(status,
                        access_token))

                # output progress occasionally to make sure code is not
                # stalling
                num_processed += 1
                if num_processed % 100 == 0:
                    print ("%s Statuses Processed: %s" % \
                        (num_processed, datetime.datetime.now()))

            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(
                                        statuses['paging']['next']))
            else:
                has_next_page = False


        print ("\nDone!\n%s Statuses Processed in %s" % \
                (num_processed, datetime.datetime.now() - scrape_starttime))


if __name__ == '__main__':
    scrapeFacebookPageFeedStatus(page_id, access_token)


# The CSV can be opened in all major statistical programs. Have fun! :)


Please Paste Public Page Name:NHL
Please Paste Your Access Token:EAACEdEose0cBALOQhAgILVZCT4Em4kH60vvE71X8yK228gj2Flj4afXCQGZCGgdOzckEQItvO4Vl6dWq9Chhlu9EkvRoorYLPbj3u3s2KY0skrSd5exVHyXiT3YpCeqfOFk6iKoAyZC7UGLqhn3AxCqyeJbYD3H6qwMDS6RWwCfgh6El6Q2DnY35fxQgOgZD
Please enter the start date (yyyy-mm-dd)2015-01-01
Please enter the end date (yyyy-mm-dd)2016-01-02
TYPE:  <class 'datetime.datetime'>
Scraping NHL Facebook Page: 2018-04-03 22:04:13.255036

100 Statuses Processed: 2018-04-03 22:04:14.364201
200 Statuses Processed: 2018-04-03 22:04:15.378465
300 Statuses Processed: 2018-04-03 22:04:16.308557
400 Statuses Processed: 2018-04-03 22:04:17.226359
500 Statuses Processed: 2018-04-03 22:04:18.049777


KeyError: 'next'

In [9]:

import json
import datetime
import csv
import time

#app_id = "<FILL IN>"
#app_secret = "<FILL IN>" # DO NOT SHARE WITH ANYONE!
file_id = input("Please Paste the Page name or Group ID:")

#access_token = app_id + "|" + app_secret
access_token = input("Please Paste Your Access Token:")

def request_until_succeed(url):
    req = requests.get(url)
    return req.text

# Needed to write tricky unicode correctly to csv
def unicode_normalize(text):
    return text.translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 
                            0x201D:0x22, 0xa0:0x20 }).encode('utf-8')

def getFacebookCommentFeedData(status_id, access_token, num_comments):

    # Construct the URL string
        base = "https://graph.facebook.com/v2.6"
        node = "/%s/comments" % status_id 
        fields = "?fields=id,message,like_count,created_time,comments,from,attachment"
        parameters = "&order=chronological&limit=%s&access_token=%s" % \
                (num_comments, access_token)
        url = base + node + fields + parameters

        # retrieve data
        data = request_until_succeed(url)
        if data is None:
            return None
        else:   
            return json.loads(data)

def processFacebookComment(comment, status_id, parent_id = ''):

    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.

    # Additionally, some items may not always exist,
    # so must check for existence first
    print(comment)
    comment_id = comment['id']
    comment_message = '' if 'message' not in comment else \
            unicode_normalize(comment['message'])
    comment_likes = 0 if 'like_count' not in comment else \
            comment['like_count']
    comment_author = 0
    if 'attachment' in comment:
        attach_tag = "[[%s]]" % comment['attachment']['type'].upper()
        comment_message = attach_tag if comment_message is '' else \
                (comment_message.decode("utf-8") + " " + \
                        attach_tag).encode("utf-8")

    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.

    comment_published = datetime.datetime.strptime(
            comment['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    comment_published = comment_published + datetime.timedelta(hours=-5) # EST
    comment_published = comment_published.strftime(
            '%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs

    # Return a tuple of all processed data

    return (comment_id, status_id, parent_id, comment_message, comment_author,
            comment_published, comment_likes)

def scrapeFacebookPageFeedComments(page_id, access_token):
    with open('%s_facebook_comments.csv' % file_id, 'w') as file:
        w = csv.writer(file)
        w.writerow(["comment_id", "status_id", "parent_id", "comment_message", 
            "comment_author", "comment_published", "comment_likes"])

        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()

        print ("Scraping %s Comments From Posts: %s\n" % \
                (file_id, scrape_starttime))

        with open('%s_facebook_statuses.csv' % file_id, 'r') as csvfile:
            reader = csv.DictReader(csvfile)

            #reader = [dict(status_id='759985267390294_1158001970921953')]

            for status in reader:
                has_next_page = True

                comments = getFacebookCommentFeedData(status['status_id'], 
                        access_token, 100)
                while has_next_page and comments is not None:				
                    for comment in comments['data']:
                        w.writerow(processFacebookComment(comment, 
                            status['status_id']))

                        if 'comments' in comment:
                            has_next_subpage = True

                            subcomments = getFacebookCommentFeedData(
                                    comment['id'], access_token, 100)

                            while has_next_subpage:
                                for subcomment in subcomments['data']:
                                    # print (processFacebookComment(
                                        # subcomment, status['status_id'], 
                                        # comment['id']))
                                    w.writerow(processFacebookComment(
                                            subcomment, 
                                            status['status_id'], 
                                            comment['id']))

                                    num_processed += 1
                                    if num_processed % 1000 == 0:
                                        print( "%s Comments Processed: %s" % \
                                                (num_processed, 
                                                    datetime.datetime.now()))

                                if 'paging' in subcomments:
                                    if 'next' in subcomments['paging']:
                                        subcomments = json.loads(
                                                request_until_succeed(
                                                    subcomments['paging']\
                                                               ['next']))
                                    else:
                                        has_next_subpage = False
                                else:
                                    has_next_subpage = False

                        # output progress occasionally to make sure code is not
                        # stalling
                        num_processed += 1
                        if num_processed % 1000 == 0:
                            print ("%s Comments Processed: %s" % \
                                    (num_processed, datetime.datetime.now()))

                    if 'paging' in comments:		
                        if 'next' in comments['paging']:
                            comments = json.loads(request_until_succeed(
                                        comments['paging']['next']))
                        else:
                            has_next_page = False
                    else:
                        has_next_page = False


        print ("\nDone!\n%s Comments Processed in %s" % \
                (num_processed, datetime.datetime.now() - scrape_starttime))


if __name__ == '__main__':
    scrapeFacebookPageFeedComments(file_id, access_token)


# The CSV can be opened in all major statistical programs. Have fun! :)


Please Paste the Page name or Group ID:NHL
Please Paste Your Access Token:EAACEdEose0cBALOQhAgILVZCT4Em4kH60vvE71X8yK228gj2Flj4afXCQGZCGgdOzckEQItvO4Vl6dWq9Chhlu9EkvRoorYLPbj3u3s2KY0skrSd5exVHyXiT3YpCeqfOFk6iKoAyZC7UGLqhn3AxCqyeJbYD3H6qwMDS6RWwCfgh6El6Q2DnY35fxQgOgZD
Scraping NHL Comments From Posts: 2018-04-03 22:04:38.910715

{'id': '10153196479592466_10153196481187466', 'message': 'Go Habs go', 'like_count': 1, 'created_time': '2016-01-01T20:53:13+0000'}
{'id': '10153196479592466_10153196481777466', 'message': 'nice goal', 'like_count': 1, 'created_time': '2016-01-01T20:53:32+0000'}
{'id': '10153196479592466_10153196481877466', 'message': 'Ahah 4-1 bye bye Boston', 'like_count': 3, 'created_time': '2016-01-01T20:53:35+0000'}
{'id': '10153196479592466_10153196481952466', 'message': 'Wow Montreal really taking it to Boston. Great game.', 'like_count': 211, 'created_time': '2016-01-01T20:53:36+0000', 'comments': {'data': [{'created_time': '2016-01-01T21:21:08+0000', 'message': 'Bruins 

{'id': '10153196479592466_10153196498027466', 'message': 'Other proof that gally should be the cap', 'like_count': 2, 'created_time': '2016-01-01T21:05:21+0000'}
{'id': '10153196479592466_10153196498487466', 'message': "Yep! He's the Heart and Soul of the Habs and always shows up even coming back from an injury.", 'like_count': 4, 'created_time': '2016-01-01T21:05:47+0000'}
{'id': '10153196479592466_10153196503712466', 'message': "Fake captain...you're an idiot..he's been our best player the last 5 years...grow up", 'like_count': 13, 'created_time': '2016-01-01T21:10:11+0000'}
{'id': '10153196479592466_10153196509442466', 'message': 'Doesn\'t show up in playoffs, nor when team is crumbling and scores 25% of his goals in empty nets. But that\'s my opinion and you have yours, I have a hard time finding a lot of fans credible because I see how so many don\'t know their hockey and only "like the Habs". I don\'t need to grow up to see the facts of who\'s a real leader and who\'s not.', 'lik

{'id': '10153196479592466_10153196519527466', 'message': 'no kidding the alumini bruins could play better than the current team', 'like_count': 1, 'created_time': '2016-01-01T21:23:38+0000'}
{'id': '10153196479592466_10153196483177466', 'message': 'Gg', 'like_count': 1, 'created_time': '2016-01-01T20:54:21+0000'}
{'id': '10153196479592466_10153196483567466', 'message': '4-1', 'like_count': 1, 'created_time': '2016-01-01T20:54:38+0000'}
{'id': '10153196479592466_10153196483612466', 'message': 'Bruins look like they are skating in quicksand.', 'like_count': 32, 'created_time': '2016-01-01T20:54:40+0000', 'comments': {'data': [{'created_time': '2016-01-01T21:42:37+0000', 'message': 'Montreal is still a faster team', 'id': '10153196479592466_10153196543302466'}, {'created_time': '2016-01-02T00:05:25+0000', 'message': 'Tim Muir Hmm, you missed the point entirely..', 'id': '10153196479592466_10153196753487466'}], 'paging': {'cursors': {'before': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEF4TlRNeE9UWTF

{'id': '10153196479592466_10153196599882466', 'message': 'Fein volley. Wie von maik gesteuert ;)', 'like_count': 0, 'created_time': '2016-01-01T22:21:04+0000'}
{'id': '10153196479592466_10153196489282466', 'message': "Johnnathan 'Civico' Vega Whats going on with ya boooooyysssss lol", 'like_count': 0, 'created_time': '2016-01-01T20:58:26+0000'}
{'id': '10153196479592466_10153196489727466', 'message': 'Great game.  Montreal is just outworking the Bs', 'like_count': 5, 'created_time': '2016-01-01T20:58:58+0000'}
{'id': '10153196479592466_10153196489907466', 'message': 'Nicholas Kennedy LJ Lauleneke Foley', 'like_count': 0, 'created_time': '2016-01-01T20:59:06+0000'}
{'id': '10153196479592466_10153196489937466', 'message': 'Nolan Johnson Gally', 'like_count': 1, 'created_time': '2016-01-01T20:59:07+0000'}
{'id': '10153196479592466_10153196490052466', 'message': "Dawson Jackson what's going on?", 'like_count': 0, 'created_time': '2016-01-01T20:59:11+0000'}
{'id': '10153196479592466_1015319

{'id': '10153196479592466_10153196500047466', 'message': 'Marchand is the real turd. Enough said.', 'like_count': 183, 'created_time': '2016-01-01T21:07:01+0000'}
{'id': '10153196479592466_10153196500512466', 'message': '4-1 this guy is maaaaaad', 'like_count': 65, 'created_time': '2016-01-01T21:07:34+0000'}
{'id': '10153196479592466_10153196502212466', 'message': "Our maybe he's just telling it as it is.", 'like_count': 0, 'created_time': '2016-01-01T21:09:08+0000'}
{'id': '10153196479592466_10153196502557466', 'message': 'You are probably jealous of him', 'like_count': 2, 'created_time': '2016-01-01T21:09:31+0000'}
{'id': '10153196479592466_10153196503427466', 'message': "Don't act like any team wouldn't want Gallagher on their team. A turd to you maybe. But you'd love him if he played for you.", 'like_count': 12, 'created_time': '2016-01-01T21:09:57+0000'}
{'id': '10153196479592466_10153196503672466', 'message': "Maybe ogre Chara should pick on someone his own size. The biggest cry 

...

{'id': '10153194634987466_10153194842997466', 'message': 'Im getting up at 500 for the game tomorrow!', 'like_count': 1, 'created_time': '2016-01-01T00:06:30+0000'}
{'id': '10153194634987466_10153194881287466', 'message': 'Good old style hockey.', 'like_count': 1, 'created_time': '2016-01-01T00:34:58+0000'}
{'id': '10153194634987466_10153194894272466', 'message': 'Jack Sea wtf this dudes heaps old aha', 'like_count': 0, 'created_time': '2016-01-01T00:41:33+0000', 'comments': {'data': [{'created_time': '2016-01-01T00:45:56+0000', 'message': 'Yeah it was the alumni game haha', 'id': '10153194634987466_10153194900342466'}, {'created_time': '2016-01-01T00:50:32+0000', 'message': 'ohhh didnt even see that ahah', 'id': '10153194634987466_10153194906202466'}, {'created_time': '2016-01-01T00:58:13+0000', 'message': 'Actual game is at 5am tomorrow morning, viewing party at mine haha', 'id': '10153194634987466_10153194918167466'}], 'paging': {'cursors': {'before': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZAN

{'id': '10153194461177466_10153194466567466', 'message': "A Price/Rask showdown would've been awesome!! Should be a great game", 'like_count': 1, 'created_time': '2015-12-31T20:02:32+0000'}
{'id': '10153194461177466_10153194467182466', 'message': 'First brawl to ever happen outdoors. Maybe...', 'like_count': 0, 'created_time': '2015-12-31T20:02:56+0000', 'comments': {'data': [{'created_time': '2015-12-31T20:03:41+0000', 'message': 'There was a fight in the 2010 winter classic', 'id': '10153194461177466_10153194468072466'}, {'created_time': '2015-12-31T20:04:55+0000', 'message': 'Like full on line brawl?', 'id': '10153194461177466_10153194469362466'}, {'created_time': '2015-12-31T20:09:51+0000', 'message': 'All the players should be in on it 😂', 'id': '10153194461177466_10153194474697466'}], 'paging': {'cursors': {'before': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEF4TlRNeE9UUTBOamd3TnpJME5qWTZANVFExTVRVNU1qSXlNUT09', 'after': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEF4TlRNeE9UUTBOelEyT1RjME5qWTZANVFExT

{'id': '10153194461177466_10153195859832466', 'message': '#Usant', 'like_count': 0, 'created_time': '2016-01-01T14:24:56+0000'}
{'id': '10153194461177466_10153195848507466', 'message': "that's the first comment I've seen cheering him on, that's great, with a solid D in front of him he'll do just fine for us today...\nGo Habs Go !!!", 'like_count': 0, 'created_time': '2016-01-01T14:18:56+0000'}
{'id': '10153194461177466_10153195852352466', 'message': 'Robert Patzak', 'like_count': 1, 'created_time': '2016-01-01T14:22:15+0000'}
{'id': '10153194461177466_10153195918542466', 'message': 'There ARE other rivalries out there you know...', 'like_count': 0, 'created_time': '2016-01-01T14:56:34+0000'}
{'id': '10153194461177466_10153196007332466', 'message': 'Sandra Gumbel Marvin Eckert Patrick Ehrenpreis Stevie Wonder 👏👏🎉🎉', 'like_count': 1, 'created_time': '2016-01-01T16:00:59+0000'}
{'id': '10153194461177466_10153196079947466', 'message': 'Love it. USA/CANADA!!!!', 'like_count': 0, 'created_ti

{'id': '10153193603212466_10153193776747466', 'message': 'Kakkahousu rangers', 'like_count': 0, 'created_time': '2015-12-31T12:18:57+0000'}
{'id': '10153193603212466_10153193673967466', 'message': 'Simon Wettstein', 'like_count': 1, 'created_time': '2015-12-31T11:03:38+0000', 'comments': {'data': [{'created_time': '2015-12-31T11:36:20+0000', 'message': 'Du bim chneble?', 'id': '10153193603212466_10153193716527466'}, {'created_time': '2015-12-31T11:37:27+0000', 'message': 'Nei ich het mitem fuess iehgspielt😂', 'id': '10153193603212466_10153193717567466'}], 'paging': {'cursors': {'before': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEF4TlRNeE9UTTNNVFkxTWpjME5qWTZANVFExTVRVMk1UYzRNQT09', 'after': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEF4TlRNeE9UTTNNVGMxTmpjME5qWTZANVFExTVRVMk1UZAzBOdz09'}}}}
{'id': '10153193603212466_10153193716527466', 'message': 'Du bim chneble?', 'like_count': 1, 'created_time': '2015-12-31T11:36:20+0000'}
{'id': '10153193603212466_10153193717567466', 'message': 'Nei ich het mitem fuess

KeyboardInterrupt: 